In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

# own Modules 
from models_sub_net_ls import LstmMse_LatentSpace, LstmMle_LatentSpace, AnalysisLayer
from data_preperator import DataPreperatorPrediction
from data_set import DataSet
from predictor import PredictorMse, PredictorMle, PredictorMseLatentSpaceAnalyser, PredictorMleLatentSpaceAnalyser

## Take care of these things before training:
- Select correct path and define droped_features
- Change parameter of model
- Change filed_location

## Parameters phm data

In [2]:
param = {
    "data" : {
        "path" : '../../data/phm_data_challenge/01_M01_DC_prediction_1.csv',
        "droped_feature" : ["stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MLE_model/phm_dataFold xx_InputSize13_LayerLstm1_HiddenLstm15_HiddenFc75_Seq25.pt",
        "input_size" : 13,
        "n_hidden_lstm" : 15,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "n_hidden_fc_pred": 75,
        "n_hidden_fc_ls": 7,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : 0.3,
        "smooth_rate" : 0.05,
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MLE/phm_mle_1.csv",
    }
}

## Parameters artifical data

In [13]:
param = {
    "data" : {
        "path" : '../../data/artifical_signals/MLE_analysis/artifical_2_signals_errors.csv',
        "droped_feature" : ["anomaly"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MLE/artifical_2_signalsFold xx_InputSize2_LayerLstm1_HiddenLstm15_HiddenFc75_Seq25.pt",
        "input_size" : 2,
        "n_hidden_lstm" : 15,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "n_hidden_fc_pred": 75,
        "n_hidden_fc_ls": 7,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : 0.3,
        "smooth_rate" : 0.05,
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MLE/MLE_1_ls_artifical_2_signals.csv",
    }
}

## Parameters cpps data

In [2]:
param = {
    "data" : {
        "path" : '../../data/cpps_data/cpps_data_predictive_maintenance.csv',
        "droped_feature" : ["status"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MLE_latent_space/artifical_2_signals_InputSize2_LayerLstm1_HiddenLstm15_HiddenFc_pred75_HiddenFc_ls7_Seq25.pt",
        "input_size" : 10,
        "n_hidden_lstm" : 15,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "n_hidden_fc_pred": 75,
        "n_hidden_fc_ls": 7,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : 0.3,
        "smooth_rate" : 0.05,
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MLE_LS/cpps_data.csv",
    }
}

## Standarize Data
First we have to apply normalisation to data. That is because the model works on the representation given by its input vectors. The scale of those numbers is part of the representation.
We should apply the exact same scaling as for training data. That means storing the scale and offset used with your training data, and using that again. <br>
__The mean and variance for each feature of the training data with which the model was trained (stake: 0.75):__

### Mean and Variance from phm Dataset
droped features="stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION"

In [55]:
mean_training_data =[ 0.21683119,  0.32121513,  0.31925213,  0.20097501,  0.45164471,  0.22914814,
  0.11604865,  0.27421592,  0.24393222, -0.13974937, -0.09739598, -0.07313758,
  0.18198089]
var_training_data =[0.75261122, 0.90482986, 0.91105327, 0.75504036, 1.07026701, 0.76708319,
 0.35172769, 0.83004988, 0.76964675, 0.57386915, 0.45912309, 0.2955709,
 1.61493449]

### Mean and Variance from artifical Dataset

In [14]:
mean_training_data= [-0.00393712, -0.01294209]
var_training_data = [49.18936568,  0.34270256]

### Mean and Variance form cpps Dataset

In [19]:
mean_training_data = [0.05330162, 0.03075699, -0.05636937, 0.0274802, 0.06536314, -0.04620979,-0.0745559, 
                      -0.08149049, -0.05318843, 0.11105582]
var_training_data = [0.02905961, 0.04473883, 0.05254194, 0.05198144, 0.07337494, 0.0666981, 0.07593811, 
                     0.0393896, 0.08028017, 0.0594492]

## Create DataLoader

In [15]:
data_preperator = DataPreperatorPrediction(path=param['data']['path'], 
                                           ignored_features = param["data"]["droped_feature"],
                                           mean_training_data=mean_training_data, 
                                           var_training_data=var_training_data, 
                                           first_order_difference=False 
                                          )                                  
preprocessed_data = data_preperator.prepare_data()
print(preprocessed_data.shape)

dataset = DataSet(preprocessed_data, 
                  timesteps=param["model"]["sequence_size"])
data_loader = DataLoader(dataset, 
                         batch_size=param['model']['batch_size'], 
                         num_workers=0, 
                         shuffle=False, 
                         drop_last=True)

(12000, 3)


In [16]:
for batch_idx, data in enumerate(data_loader):
    x,y = data
    print('Data of batch: {}'.format(batch_idx))
    print("Size of input data: {}".format(x.size()))
    print("Size of target data: {}".format(y.size()))
    if batch_idx >=1: break

Data of batch: 0
Size of input data: torch.Size([50, 100, 3])
Size of target data: torch.Size([50, 3])
Data of batch: 1
Size of input data: torch.Size([50, 100, 3])
Size of target data: torch.Size([50, 3])


## Define Model and load Parameters of trained model
### Model for MSE and Latent Space Analysis

In [ ]:
model = LstmMse_LatentSpace(batch_size=param['model']['batch_size'], 
                            input_dim=param['model']['input_size'], 
                            n_hidden_lstm=param['model']['n_hidden_lstm'], 
                            n_layers=param['model']['lstm_layer'],
                            dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                            dropout_rate_fc= param['model']['dropout_rate_fc'],
                            n_hidden_fc_prediction=param['model']['n_hidden_fc_pred'], 
                            n_hidden_fc_ls_analysis=param['model']['n_hidden_fc_ls']      
                            )

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

### Model for MLE and Latent Space Analysis

In [6]:
model = LstmMle_LatentSpace(batch_size=param['model']['batch_size'], 
                            input_dim=param['model']['input_size'], 
                            n_hidden_lstm=param['model']['n_hidden_lstm'], 
                            n_layers=param['model']['lstm_layer'],
                            dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                            dropout_rate_fc= param['model']['dropout_rate_fc'],
                            n_hidden_fc_prediction=param['model']['n_hidden_fc_pred'], 
                            n_hidden_fc_ls_analysis=param['model']['n_hidden_fc_ls'],
                            K = param['model']['K']
                            )

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

## Initialize Predictor
### Predictor for MSE Model and Latent Space Analysis

In [ ]:
predictor = PredictorMseLatentSpaceAnalyser(model=model,
                                            path_data=param["data"]["path"],
                                            columns_to_ignore=param["data"]["droped_feature"],
                                            threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"]
                                            )

### Predictor for MLE Model and Latent Space Analysis

In [21]:
predictor = PredictorMleLatentSpaceAnalyser(model=model,
                                            path_data=param["data"]["path"],
                                            columns_to_ignore=param["data"]["droped_feature"],
                                            threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"],
                                            no_standard_deviation=param["anomaly_detection"]["no_standard_deviation"]
                                            )

## Predict

In [22]:
print("Start predicting.")
# Write header
with open(param["results"]["path"], "a+") as file:
            [file.write(column+";") for column in predictor.create_column_names_result()]
            file.write("\n")

for batch_number, (input_data, target_data) in enumerate(data_loader):
    # Predict sensor values in mini-batches
    batch_results = predictor.predict(input_data, target_data)
    
    # Write results to csv file
    with open(param["results"]["path"], "a") as file:
        for batch in batch_results:
            # Each result component of a singe prediction (ID, target, prediction, loss, latent space ...) is stored in lists
            # thus we have to unpack the list and seperate values with ;
            for value in batch:
                file.write(str(value)+";")
            file.write("\n")

    # Print status 
    if (batch_number*param['model']['batch_size'])%5000 == 0:
        print("Current status: " + str(param['model']['batch_size']*batch_number) + " samples are predicted.")

print("End of prediction.")

Start predicting.


AttributeError: 'LstmMle_1' object has no attribute 'n_hidden_fc_ls_analysis'

## Tag anomalous samples

In [9]:
results_prediction = pd.read_csv(param["results"]["path"], sep=";")
# Values of column "loss" are exponentially smoothed and stored in a new column "smoothed loss"
# New column "anomaly" is created and sample is taged with 1 if anomalous behaviour (if smoothed loss is over threshold)
results = predictor.detect_anomaly(results_prediction, param["anomaly_detection"]["smooth_rate"])

In [10]:
results.head()

,ID,sine_signal target,sawtooth_signal target,sine_signal mu predicted,sawtooth_signal mu predicted,sine_signal sigma predicted,sawtooth_signal sigma predicted,mean normalised residual,sine_signal normalised residual,sawtooth_signal normalised residual,latent_space_0,latent_space_1,latent_space_2,latent_space_3,latent_space_4,latent_space_5,latent_space_6,Anomaly Sensor_1,Anomaly Sensor_2
0,100.0,0.129967,-1.543861,-0.062869,-0.888886,0.095342,1.233198,0.745722,2.022563,-0.531119,0.444841,-0.556853,0.240179,-0.281422,-0.414726,-0.235273,0.220986,1,0
1,101.0,0.132623,-1.655911,0.158563,-1.478936,0.087534,0.229338,-0.534012,-0.296344,-0.771681,0.126500,-0.446662,0.316357,-0.080894,-0.858233,-0.312968,0.147232,0,0
2,102.0,0.378911,-1.290781,0.342904,-1.339003,0.085570,0.204115,0.328521,0.420794,0.236248,0.094590,-0.372622,0.281857,-0.011506,-0.808945,-0.373254,0.097531,0,0
3,103.0,0.383205,-1.221328,0.527406,-1.159326,0.085201,0.195283,-1.004999,-1.692496,-0.317503,0.057978,-0.276071,0.253816,-0.000141,-0.745135,-0.427654,0.068529,0,0
4,104.0,0.658533,-0.668286,0.645150,-0.966852,0.085493,0.197475,0.834228,0.156541,1.511915,0.025026,-0.192565,0.209690,0.015146,-0.667188,-0.476022,0.048455,0,0


## Combine prediction data with data which was not consider for inference

In [11]:
original_sensor_data = pd.read_csv(param["data"]["path"])
data_of_droped_feature = original_sensor_data.loc[:, param["data"]["droped_feature"]+["ID"]]
complete_data = results.merge(right=data_of_droped_feature, how="inner", on="ID")

## Save data to csv file

In [12]:
complete_data.to_csv(param["results"]["path"], sep=";", index=False)